In [1]:
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html
!pip install transformers
!pip install pytorch_lightning
!pip install sentencepiece

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
     |████████████████████████████████| 895 kB 69.0 MB/s 
     |████████████████████████████████| 6.5 MB 3.2 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 527 kB 4.3 MB/s 
     |████████████████████████████████| 397 kB 54.3 MB/s 
     |████████████████████████████████| 952 kB 89.1 MB/s 
     |████████████████████████████████| 829 kB 51.7 MB/s 
     |████████████████████████████████| 134 kB 29.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.6 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 144 kB 94.3 MB/s 
     |████████████████████████████████| 271 kB 68.8 MB/s 
  Created wheel for future: filename=future-0.

     |████████████████████████████████| 1.2 MB 4.1 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive/')
root = "drive/MyDrive/lign167_data"

Mounted at /content/drive/


In [ ]:
from torch.utils.data import Dataset , DataLoader
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
from itertools import chain

In [ ]:
from transformers.models.bart.modeling_bart import BartEncoder, BartDecoder, BartPretrainedModel
from transformers.modeling_outputs import (BaseModelOutput,
    BaseModelOutputWithPastAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput,
    Seq2SeqQuestionAnsweringModelOutput,
    Seq2SeqSequenceClassifierOutput,)
from transformers.models.bart.configuration_bart import BartConfig
from transformers.models.bart import BartTokenizerFast
# BartEncoder.from_pretrained("facebook/bart-base")
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")

https://huggingface.co/facebook/bart-base/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpwfdeq25w


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
creating metadata file for /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
https://huggingface.co/facebook/bart-base/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpfn9tqo16


Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/facebook/bart-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7c1p7wcv


Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/a878fcd69bba037c9b1b227f4213579ae43d0aaa9374e167bc6c5f41b1cfeb30.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/a878fcd69bba037c9b1b227f4213579ae43d0aaa9374e167bc6c5f41b1cfeb30.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/facebook/bart-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/facebook/bart-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://hugg

In [ ]:
def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    print(input_ids)
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    if pad_token_id is None:
        raise ValueError("self.model.config.pad_token_id has to be defined.")
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids

class BartModel(BartPretrainedModel):
    def __init__(self, config: BartConfig):
        super().__init__(config)

        padding_idx, vocab_size = config.pad_token_id, config.vocab_size
        self.shared = nn.Embedding(vocab_size, config.d_model, padding_idx)

        self.encoder = BartEncoder(config, self.shared)
        self.extractor = BartEncoder(config, self.shared)
        self.decoder = BartDecoder(config, self.shared)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.shared

    def set_input_embeddings(self, value):
        self.shared = value
        self.encoder.embed_tokens = self.shared
        self.decoder.embed_tokens = self.shared

    def get_encoder(self):
        return self.encoder

    def get_decoder(self):
        return self.decoder

    def forward(
        self,
        input_ids=None,
        context_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        encoder_outputs=None,
        extractor_outputs=None,
        past_key_values=None,
        inputs_embeds=None,
        context_embeds=None,
        decoder_inputs_embeds=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):

        # different to other models, Bart automatically creates decoder_input_ids from
        # input_ids if no decoder_input_ids are provided
        if decoder_input_ids is None and decoder_inputs_embeds is None:
            if input_ids is None:
                raise ValueError(
                    "If no `decoder_input_ids` or `decoder_inputs_embeds` are "
                    "passed, `input_ids` cannot be `None`. Please pass either "
                    "`input_ids` or `decoder_input_ids` or `decoder_inputs_embeds`."
                )
            if context_ids is None:
                raise ValueError(
                    "If no `decoder_input_ids` or `decoder_inputs_embeds` are "
                    "passed, `context_ids` cannot be `None`. Please pass either "
                    "`context_ids` or `decoder_input_ids` or `decoder_inputs_embeds`."
                )

            decoder_input_ids = shift_tokens_right(
                input_ids, self.config.pad_token_id, self.config.decoder_start_token_id
            )

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if encoder_outputs is None:
            encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
        # If the user passed a tuple for encoder_outputs, we wrap it in a BaseModelOutput when return_dict=True
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
            )
        

        if extractor_outputs is None:
            extractor_outputs = self.encoder(
                input_ids=context_ids,
                attention_mask=attention_mask,
                head_mask=head_mask,
                inputs_embeds=context_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
        # If the user passed a tuple for encoder_outputs, we wrap it in a BaseModelOutput when return_dict=True
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            extractor_outputs = BaseModelOutput(
                last_hidden_state=extractor_outputs[0],
                hidden_states=extractor_outputs[1] if len(extractor_outputs) > 1 else None,
                attentions=extractor_outputs[2] if len(extractor_outputs) > 2 else None,
            )

        # decoder outputs consists of (dec_features, past_key_value, dec_hidden, dec_attn)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            encoder_hidden_states=encoder_outputs[0] + extractor_outputs[0],
            encoder_attention_mask=attention_mask,
            head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if not return_dict:
            return decoder_outputs + encoder_outputs

        return Seq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )

In [ ]:
model = BartModel.from_pretrained("facebook/bart-base").cuda()

loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.da0f3c0e2dc1c2fecc46738a1ebf4806f2fc36aae3d5c1947f21e063e7cab34b
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false

In [ ]:
ex_input = "I like this place so much <style-pos>"
ex_context = "Aha I hate it"
print(ex_input)
ex_input_ids = tokenizer(ex_input, max_length=32, truncation=True, padding="max_length", return_tensors="pt").input_ids.cuda()
ex_context_ids = tokenizer(ex_context, max_length=32, truncation=True, padding="max_length", return_tensors="pt").input_ids.cuda()
model(input_ids=ex_input_ids, context_ids=ex_context_ids)

I like this place so much <style-pos>
tensor([[    0,   100,   101,    42,   317,    98,   203, 28696,  5827,    12,
         11474, 15698,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1]], device='cuda:0')


Seq2SeqModelOutput([('last_hidden_state',
                     tensor([[[ 2.2926,  1.7926,  1.4142,  ...,  1.4553,  0.2918,  0.0241],
                              [-0.2750, -0.6829, -0.9581,  ...,  1.8924, -1.1731,  1.9685],
                              [ 0.4447,  0.1336, -0.7446,  ..., -0.2573,  0.2131,  0.7513],
                              ...,
                              [ 0.0953,  0.0343,  1.8077,  ...,  0.3308,  0.9270,  0.2682],
                              [ 0.0995,  0.0970,  1.8980,  ...,  0.2583,  0.9205,  0.2891],
                              [ 0.1156,  0.1557,  1.9473,  ...,  0.2385,  0.8814,  0.3276]]],
                            device='cuda:0', grad_fn=<NativeLayerNormBackward0>)),
                    ('past_key_values',
                     ((tensor([[[[-1.1472e-01, -5.2203e-01, -6.7271e-01,  ...,  3.4312e-01,
                                   7.5881e-02, -2.4253e-02],
                                 [ 5.4507e-02,  3.6397e-01, -7.1711e-01,  ...,  8.1498e-02,
 

In [ ]:
raw_data = pd.concat([pd.read_csv(f"{root}/yelp/pos.csv", sep="\t"),pd.read_csv(f"{root}/yelp/neg.csv", sep="\t")])
raw_data["0"] += " <style-" + raw_data["tag"] + ">"
raw_data

,0,tag
0,these donuts have the perfect texture and tast...,pos
1,good food for the price . <style-pos>,pos
2,"a little dirty on the inside , but wonderful p...",pos
3,i always order it when i go there and it is al...,pos
4,the rest of the food there is good also and no...,pos
...,...,...
180213,this place sucks . <style-neg>,neg
180214,does n't stop by to see if you need anything ....,neg
180215,food is terrible . <style-neg>,neg
180216,service horrible . <style-neg>,neg


In [ ]:
class YelpDataset(Dataset):
    def __init__(self,data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def to_token(self,sentence):
        return tokenizer.encode(sentence, max_length=32, truncation=True, padding="max_length", return_tensors="pt")

    def __getitem__(self,index):
        tokens = self.to_token(self.data["0"].iloc[index])[0]
        return tokens, self.data["tag"].iloc[index]

In [ ]:
train_dataset, val_dataset = train_test_split(raw_data, test_size=0.2)
batch_size = 16

In [ ]:
class YelpDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train = YelpDataset(train_dataset)
        self.test = YelpDataset(val_dataset)
        self.val = YelpDataset(val_dataset)

    def train_dataloader(self):
        return DataLoader(self.train , batch_size = batch_size , shuffle = True, num_workers=4)
    def test_dataloader(self):
        return DataLoader(self.test , batch_size = batch_size , shuffle = False, num_workers=4)
    def val_dataloader(self):
        return DataLoader(self.val , batch_size = batch_size , shuffle = False, num_workers=4)

In [ ]:
labels = ['neg', 'pos']

In [ ]:
from transformers import BartTokenizerFast, BartForConditionalGeneration, BartConfig

eos_token_id = 2
config = BartConfig.from_pretrained("facebook/bart-base", output_hidden_states=True, eos_token_id = eos_token_id)
bart_generator = BartForConditionalGeneration.from_pretrained("facebook/bart-base", config=config)

tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")
tokenizer.add_tokens([f"<style-{l}>" for l in labels])
bart_generator.resize_token_embeddings(len(tokenizer))

Embedding(50267, 768)

'<s>'

In [ ]:
def getLabelID(l):
  if isinstance(l, int):
    l = labels[l]
  return tokenizer(f"<style-{l}>")["input_ids"][1]

label2TokenID = dict(zip(labels, map(lambda l : getLabelID(l), labels)))
def getLabelToken(l):
  if isinstance(l, torch.Tensor):
    l = l.item()
  if isinstance(l, int):
    l = labels[l]
  return label2TokenID[l]

style_token_base = getLabelToken(labels[0])
style_token_base

50265

In [ ]:
### UTILS

labels = ['neg', 'pos']
def label_to_index(label):
  return labels.index(label)

def label_to_indices(ls):
  return torch.tensor([label_to_index(l) for l in ls])

def random_labels(n = batch_size):
  return np.random.choice(labels, n)

def random_label_indices(n = batch_size):
  return torch.tensor(np.random.choice(np.arange(0, len(labels)), n))

ex_labels = random_label_indices()
ex_sents = torch.tensor([[    0,    29,   625,   352,  2156,  4146,     9,    14,    16,  1528,
           122,   479,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   118,    33,  1240,    10,   319,     9,    86,    23,  1094,
         16296,   258, 20317,     8,    66,   479,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 12338,   138, 25606,    33,   341,   106,    13,   484,   107,
           479,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 10010,    32,     5,   275,   479,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,  5016,  2512,    10,  4402,     8,  1095,    25,   444,   409,
            31,    42,   317,    25,   678,   479,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   627,    82,    32,  2579,  2156,  1365,   164,   479,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   627, 22126,    21,  1256,   739,     8,     5,  8929,    21,
           182,   205,   479,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 14668,    21,    45,  7678,  1075,  1630,  1295,    50, 23842,
         15557,    23,    70,    13, 21025,    84,    86,   479,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 24382,   118,    24,    16,   295,    75,     5,   455,  6664,
         21746,   128,    29,  5765,    98,   103, 11929,    32,  1716,   479,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   405,   128,    29,   460,  6921, 18469,   479,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,  2126,  1049, 10230,    58,  6681, 28368,   479,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,  3463, 39462,  7080,   425,     8,   689,   479,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   118, 19958,   295,    75,    28,   124,   479,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 12338,  3757,   317,     7,  3529,   479,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0, 10010,   109,   295,    75,   109,   295,    75,   190,   582,
          1503,     7,    99,    47,   128,   241, 12926, 27785,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   627,   689,    21, 28368,   479,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1]])

# ex_sents > 4 


In [ ]:
d_model = 768

class Discriminator(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(
        self,
        input_dim: int,
        inner_dim: int,
        num_classes: int,
        pooler_dropout: float,
    ):
        super().__init__()
        self.dense = nn.Linear(input_dim, inner_dim)
        self.dropout = nn.Dropout(p=pooler_dropout)
        self.out_proj = nn.Linear(inner_dim, num_classes)

    def forward(self, hidden_states: torch.Tensor):
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.dense(hidden_states)
        hidden_states = torch.tanh(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.out_proj(hidden_states)
        hidden_states = F.log_softmax(hidden_states)
        return hidden_states

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        bart_generator.train()
        self.generator = bart_generator
      
    def add_mask(self, sents, prob=0.8):
      sents[(sents.cuda() > 4) & (torch.cuda.FloatTensor(sents.shape[0], sents.shape[1]).uniform_() > prob)] = tokenizer.mask_token_id

    def forward(self, x):
      l = x.clone()
      l[(l == getLabelToken("neg")) | (l == getLabelToken("pos"))] = 1
      i = x.clone()
      self.add_mask(i)
      return self.generator(input_ids=i, labels=l)

In [ ]:
from pytorch_lightning.core.lightning import LightningModule
class StyleTransformer(LightningModule):
  def __init__(self):
    super().__init__()
    self.generator = Generator()
    self.discriminator = Discriminator(d_model, d_model, 2, 0.0)
    
    self._init_weights(self.discriminator.dense)
    self._init_weights(self.discriminator.out_proj)
  
  def _init_weights(self, module):
        std = 0.02
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()

  def adversarial_loss(self, labels_hat, adv_labels):
    # print("labels_hat:", torch.argmax(labels_hat, axis=1))
    # print("labels_hat:", labels_hat.shape)
    # print("adv_labels:", adv_labels)
    return F.cross_entropy(labels_hat, adv_labels)

  def discriminator_loss_step(self, sentences, sentences_hat, labels):
    hidden_states = sentences_hat.decoder_hidden_states[-1] # last decoder hidden state
    # print(hidden_states.shape)
      
    eos_mask = (sentences == getLabelToken("neg")) | (sentences == getLabelToken("pos"))
    # print(sentences)
    # print(eos_mask)
    if len(torch.unique_consecutive(eos_mask.sum(1))) > 1:
          raise ValueError("All examples must have the same number of <eos> tokens.")
    torch.roll(eos_mask, shifts=(0, -1), dims=(0, 1))
    # TODO!!!! MOVE STYLE LABEL TO THE END OF THE SENTENCE SO THAT EOS STATE WON"T LOOK AT IT
    # eos_mask[:,0] = True
    # bart_generator.vocab_size
    sentence_representation = hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[:, -1, :]
    # sentence_representation.detach()
    # logits = sentences_hat.logits
    # sentence_representation = logits.view(logits.size(0), -1, logits.size(-1))[:, -1, :]
    # print(sentences_hat.logits.shape)
    # print(sentence_representation.shape)
    outputs = self.discriminator(sentence_representation)
    d_loss = self.adversarial_loss(outputs, labels)
    return d_loss

  def append_style_token(self, sentences, labels):
    return torch.concat((labels.view(-1, 1) + style_token_base, sentences), dim = 1)
    
  def training_step(self, batch, batch_idx):
    sentences, labels_text = batch[0], batch[1]
    labels = label_to_indices(labels_text).to(self.device)
    # generated_labels = random_label_indices().to(self.device)
    invert_labels = 1 - labels

    orig_sentences = sentences
    
    invt_sentences = sentences.clone()
    invt_sentences[sentences == getLabelToken("neg")] = getLabelToken("pos")
    invt_sentences[sentences == getLabelToken("pos")] = getLabelToken("neg")

    # # train generator
    # if optimizer_idx == 0:
    sentences_hat = self.generator(orig_sentences)
    g_loss = sentences_hat.loss
    # if batch_idx < self.trainer.num_training_batches * 0.1:
    #   return None
    self.log("g_loss", g_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    # train discriminator
    # if optimizer_idx == 1:
    d_loss = 0

    # sentences_hat = self.generator(orig_sentences)
    d_loss += self.discriminator_loss_step(orig_sentences, sentences_hat, labels) / 2

    # if batch_idx < self.trainer.num_training_batches * 0.1:
    sentences_inv = self.generator(invt_sentences)
    d_loss += self.discriminator_loss_step(invt_sentences, sentences_inv, invert_labels) / 2
    
    self.log("d_loss", d_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
      # return d_loss
  
    return g_loss + d_loss * 10
  
  # def backward(self, loss, optimizer, optimizer_idx):
  #   loss.backward(retain_graph=True)

  def configure_optimizers(self):
        opt_d = torch.optim.AdamW(chain(self.discriminator.parameters(), self.generator.parameters()), 1e-4)
        return opt_d

In [ ]:
from pytorch_lightning import Trainer

model = StyleTransformer()
module = YelpDataModule()
trainer = Trainer(max_epochs = 10, gpus=1, default_root_dir=root)
trainer.fit(model,module)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 139 M 
1 | discriminator | Discriminator | 592 K 
------------------------------------------------
140 M     Trainable params
0         Non-trainable params
140 M     Total params
560.056   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
# orig_sample = val_dataset.sample()["original"].iloc[0]
orig_sample = "I like this place so much <style-pos>"
print(orig_sample)
input_ids = tokenizer(orig_sample, return_tensors="pt").input_ids.cuda()
outputs = model.generator.generator.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
outputs